# Lab 1: LangChain Agents

**Objective:** Build a research assistant agent with custom tools, memory, and debugging capabilities.

**Duration:** ~45 minutes

**What You'll Learn:**
- How to create custom tools using the `@tool` decorator
- How to build agents with the ReAct pattern
- How to add conversational memory
- How to debug agent behavior

## Part 1: Setup and Imports

First, let's import the required libraries and configure our environment.

In [ ]:
# Environment setup
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Verify API keys
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found in environment"
print("Environment configured successfully!")

In [ ]:
# Core imports
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain import hub
from langchain.memory import ConversationBufferMemory

print("Imports successful!")

## Part 2: Understanding Tools

Tools are functions that agents can call to interact with the outside world. In LangChain, we use the `@tool` decorator to define them.

In [ ]:
# Simple calculator tool
@tool
def calculator(expression: str) -> str:
    """Evaluates a mathematical expression and returns the result.
    Use this for any math calculations.
    
    Args:
        expression: A mathematical expression like '2 + 2' or '15 * 7'
    """
    try:
        # Only allow safe mathematical operations
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error evaluating expression: {str(e)}"

# Test the tool
print(calculator.invoke("15 * 7 + 3"))

In [ ]:
# Current date/time tool
from datetime import datetime

@tool
def get_current_datetime() -> str:
    """Returns the current date and time. Use this when you need to know what time or date it is."""
    now = datetime.now()
    return f"Current date and time: {now.strftime('%Y-%m-%d %H:%M:%S')}"

# Test the tool
print(get_current_datetime.invoke(""))

In [ ]:
# Web search tool using Tavily
from langchain_community.tools.tavily_search import TavilySearchResults

# Check if Tavily API key is available
if os.getenv("TAVILY_API_KEY"):
    search_tool = TavilySearchResults(
        max_results=3,
        description="Search the web for current information. Use this when you need up-to-date information."
    )
    print("Tavily search tool configured!")
else:
    # Fallback mock search for demo purposes
    @tool
    def search_tool(query: str) -> str:
        """Search the web for information (mock version)."""
        return f"Mock search results for: {query}. In production, configure TAVILY_API_KEY for real search."
    print("Using mock search tool (set TAVILY_API_KEY for real search)")

## Part 3: Building Your First Agent

Now let's build an agent that can use these tools.

In [ ]:
# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0  # Lower temperature for more consistent behavior
)

# Define our tools list
tools = [calculator, get_current_datetime, search_tool]

print(f"Configured {len(tools)} tools:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description[:50]}...")

In [ ]:
# Get the ReAct prompt from LangChain hub
prompt = hub.pull("hwchase17/react")

# Let's examine the prompt structure
print("ReAct Prompt Template:")
print("=" * 50)
print(prompt.template[:500] + "...")

In [ ]:
# Create the ReAct agent
agent = create_react_agent(llm, tools, prompt)

# Wrap in an executor with verbose mode for debugging
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Shows the agent's thought process
    max_iterations=5,  # Prevent infinite loops
    handle_parsing_errors=True
)

print("Agent created successfully!")

In [ ]:
# Test the agent with a simple query
response = agent_executor.invoke({
    "input": "What is 15 multiplied by 7, and what time is it right now?"
})

print("\n" + "=" * 50)
print("Final Answer:", response["output"])

### Understanding the Agent's Reasoning

Notice how the agent:
1. **Thought** about what tools it needs
2. **Acted** by calling the appropriate tool
3. **Observed** the result
4. **Repeated** until it had all the information
5. **Responded** with the final answer

This is the **ReAct** (Reason + Act) pattern in action!

## Part 4: Adding Custom Tools

Let's create more sophisticated custom tools.

In [ ]:
# Weather tool (simulated)
@tool
def get_weather(city: str) -> str:
    """Get the current weather for a city.
    
    Args:
        city: The name of the city to get weather for
    """
    # In production, this would call a real weather API
    import random
    conditions = ["sunny", "cloudy", "rainy", "partly cloudy"]
    temp = random.randint(50, 85)
    condition = random.choice(conditions)
    return f"Weather in {city}: {temp}°F, {condition}"

# Test it
print(get_weather.invoke("San Francisco"))

In [ ]:
# Note-taking tool with state
notes_storage = []

@tool
def save_note(note: str) -> str:
    """Save a note for later reference.
    
    Args:
        note: The note content to save
    """
    notes_storage.append(note)
    return f"Note saved! You now have {len(notes_storage)} note(s)."

@tool
def get_notes() -> str:
    """Retrieve all saved notes."""
    if not notes_storage:
        return "No notes saved yet."
    return "Saved notes:\n" + "\n".join(f"- {note}" for note in notes_storage)

# Test
print(save_note.invoke("Remember to review LangChain docs"))
print(get_notes.invoke(""))

In [ ]:
# Create an enhanced agent with all tools
enhanced_tools = [calculator, get_current_datetime, search_tool, get_weather, save_note, get_notes]

enhanced_agent = create_react_agent(llm, enhanced_tools, prompt)

enhanced_executor = AgentExecutor(
    agent=enhanced_agent,
    tools=enhanced_tools,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

print(f"Enhanced agent with {len(enhanced_tools)} tools ready!")

In [ ]:
# Test the enhanced agent
response = enhanced_executor.invoke({
    "input": "What's the weather in New York? Save a note about it."
})

print("\n" + "=" * 50)
print("Final Answer:", response["output"])

## Part 5: Adding Memory

Agents become more useful when they can remember previous interactions.

In [ ]:
# Get the conversational ReAct prompt
conversational_prompt = hub.pull("hwchase17/react-chat")

print("Conversational prompt loaded!")
print("\nInput variables:", conversational_prompt.input_variables)

In [ ]:
# Create memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create conversational agent
conversational_agent = create_react_agent(llm, enhanced_tools, conversational_prompt)

conversational_executor = AgentExecutor(
    agent=conversational_agent,
    tools=enhanced_tools,
    memory=memory,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

print("Conversational agent with memory ready!")

In [ ]:
# First interaction
response1 = conversational_executor.invoke({
    "input": "My name is Alice. What's the weather in Seattle?"
})
print("\nResponse 1:", response1["output"])

In [ ]:
# Second interaction - the agent should remember the name
response2 = conversational_executor.invoke({
    "input": "What's my name? And save a note that says I'm interested in Seattle weather."
})
print("\nResponse 2:", response2["output"])

In [ ]:
# Third interaction - check memory and notes
response3 = conversational_executor.invoke({
    "input": "What notes do we have? Summarize our conversation."
})
print("\nResponse 3:", response3["output"])

## Part 6: Debugging and Best Practices

Let's explore techniques for debugging and improving agent behavior.

In [ ]:
# Enable LangChain debugging
from langchain.globals import set_debug, set_verbose

# Uncomment to enable detailed debugging
# set_debug(True)
set_verbose(True)

print("Debug settings configured")

In [ ]:
# Custom error handling tool
@tool
def risky_operation(data: str) -> str:
    """A tool that might fail - for demonstrating error handling.
    
    Args:
        data: Input data to process
    """
    if "error" in data.lower():
        raise ValueError("Simulated error for demonstration")
    return f"Successfully processed: {data}"

# The agent with handle_parsing_errors=True will gracefully handle this
print("Risky operation tool created")

In [ ]:
# Best practice: Create well-documented tools
@tool
def analyze_sentiment(text: str) -> str:
    """Analyze the sentiment of a given text.
    
    This tool examines text and determines if the overall sentiment
    is positive, negative, or neutral. Use this when you need to
    understand how someone feels about something based on their writing.
    
    Args:
        text: The text to analyze for sentiment
        
    Returns:
        A string describing the sentiment (positive, negative, or neutral)
        along with confidence indicators.
    """
    # Simplified sentiment analysis
    positive_words = ['good', 'great', 'excellent', 'happy', 'love', 'amazing', 'wonderful']
    negative_words = ['bad', 'terrible', 'awful', 'sad', 'hate', 'horrible', 'disappointing']
    
    text_lower = text.lower()
    pos_count = sum(1 for word in positive_words if word in text_lower)
    neg_count = sum(1 for word in negative_words if word in text_lower)
    
    if pos_count > neg_count:
        return f"Sentiment: POSITIVE (found {pos_count} positive indicators)"
    elif neg_count > pos_count:
        return f"Sentiment: NEGATIVE (found {neg_count} negative indicators)"
    else:
        return "Sentiment: NEUTRAL (no strong indicators found)"

# Test
print(analyze_sentiment.invoke("I love this amazing product! It's wonderful."))

## Challenge: Build Your Own Research Agent

Now it's your turn! Create a research assistant agent with the following capabilities:

1. **Web search** - Find current information
2. **Note taking** - Save important findings
3. **Calculator** - Perform calculations
4. **Summary tool** - Summarize findings

The agent should be able to:
- Research a topic
- Save key findings as notes
- Perform any needed calculations
- Provide a summary

In [ ]:
# TODO: Create a summary tool
@tool
def summarize_research() -> str:
    """Summarize all research findings from saved notes.
    
    Use this after gathering information to create a summary
    of all saved notes.
    """
    # TODO: Implement this
    # Hint: Use the notes_storage list
    pass

# TODO: Create your research agent with all the tools
# research_tools = [...]
# research_agent = create_react_agent(...)
# research_executor = AgentExecutor(...)

In [ ]:
# TODO: Test your research agent
# Example query: "Research the current state of AI in healthcare. 
#                 Save the key findings and provide a summary."

## Lab Summary

In this lab, you learned:

1. **Tool Creation**: Using `@tool` decorator to create agent tools
2. **ReAct Pattern**: How agents reason and act in a loop
3. **Agent Building**: Creating agents with `create_react_agent`
4. **Memory**: Adding conversational memory with `ConversationBufferMemory`
5. **Debugging**: Using verbose mode and debugging tools

### Key Takeaways

- Good tool descriptions are **critical** - they help the agent understand when to use each tool
- Use `verbose=True` during development to understand agent behavior
- Set `max_iterations` to prevent infinite loops
- Memory enables multi-turn conversations
- Error handling (`handle_parsing_errors=True`) makes agents more robust

### Next Steps

In Lab 2, you'll learn about multi-agent systems with AutoGen and CrewAI!

In [ ]:
# Cleanup
set_verbose(False)
print("Lab 1 complete!")